# Matrix Generator for FEM
Given a certain differntial equation with boundary value condition, solve the the system using the alrogithm ??

file:///G:/My%20Drive/HKA%20EITB/Semester%206/Methoden%20der%20Feldberechnung/GalerkinVerfahren.pdf, page 13

## Task
Differential Equation:
$$
-\frac{d}{d x}\left(x^2 \frac{d \Phi}{d x}\right)+x \Phi=-x^3
$$

boundary value condition:
$$
\Phi(x=1)=2\\ \Phi(x=2)=6
$$

domain knots:
$$
[1.75, 2, 1.25, 1, 1.5]
$$

In [138]:
import numpy as np
import random
import math
import sys

## Express the given values and functions

In [139]:
def alpha(x):
    "given in the diff. eq."
    return np.square(x)

def beta(x):
    "given in the diff. eq."
    return x

def right_hand_side(x):
    return - np.power(x, 3)

# given from the boundary conditions:
# LOWER_BOUND = 1
# UPPER_BOUND = 2

# NUMBER_OF_DOMAINS = 5

In [140]:
domain_knots = np.array([1.75, 2, 1.25, 1, 1.5], dtype=float)
domain_knots

array([1.75, 2.  , 1.25, 1.  , 1.5 ])

## Create the subdomain index list

In [154]:
def get_edge_indices(knots: np.array) -> np.array:
    "Given a 1D Array of the domain points, calculate the indices of each segment between two points. the input array can be in random order. Returns an array, each element is a segment and contains the start and end index."
    sorted = np.sort(knots)

    # assume 1d, so each segment can be desribed by 2 points. 
    indices = np.zeros(shape=(sorted[:-1].size, 2), dtype=int)

    # ignore the last value in the sorted data, number of segments = number of points -1.
    for index, sorted_knot in enumerate(sorted[:-1]):

        start_index = np.where(knots == sorted_knot)[0][0]
        end_index = np.where(knots == sorted[index +1])[0][0]

        indices[index] = int(start_index), int(end_index)
    return indices

domain_edge_indices = get_edge_indices(domain_knots)
domain_edge_indices

array([[3, 2],
       [2, 4],
       [4, 0],
       [0, 1]])

## calculate matrix values
Calculate matrix values according to the following equations:
$$
L_E = x_2 -x_1
$$

$$
x_M = \frac{x_2 + x_1}{2}
$$

$$
K_{i,j} = \frac{\alpha{x_M}}{L_E} \begin{pmatrix} 1 & -1 \\ -1 & 1 \end{pmatrix} + \frac{L_E \cdot \beta{x_M}}{6} \begin{pmatrix} 2 & 1 \\ 1 & 2 \end{pmatrix}
$$

$$
D_j = \frac{L_E \cdot f{x_M}}{2} \begin{pmatrix} 1& 1 \end{pmatrix}
$$

In [152]:
def coefficients(midpoint, edge_length, alpha, beta):
    matrix1 = np.array([1, -1, -1, 1]).reshape(2,2)
    matrix2 = np.array([2, 1, 1, 2]).reshape(2,2)

    return alpha(midpoint) / edge_length * matrix1 + edge_length * beta(midpoint) / 6 * matrix2

def d(midpoint, edge_length, right_hand_side):
    matrix = np.array([1,1])
    return edge_length / 2 * right_hand_side(midpoint) * matrix

In [ ]:
for edge_index in domain_edge_indices:
    start_index, end_index = edge_index
    print(start_index, end_index)

    start_point = domain_knots[start_index]
    end_point = domain_knots[end_index]
    print(start_point, end_point)

    edge_length = end_point - start_point
    print(edge_length)

    edge_midpoint = (start_point + end_point) / 2
    print(edge_midpoint)

    alpha_value = alpha(edge_midpoint)
    beta_value = beta(edge_midpoint)
    right_hand_side_value = right_hand_side(edge_midpoint)
    print(alpha_value, beta_value, right_hand_side_value)

    k_11 = coefficients(edge_midpoint, edge_length, alpha, beta)
    print(k_11)

    d_1 = d(edge_midpoint, edge_length, right_hand_side)
    print(d_1)

    print(f'''
edge index:\t{edge_index}
edges:\t{start_point, end_point}
edge length:\t{edge_length}
edge midpoint:\t{edge_midpoint}
alpha of midpoint:\t{alpha_value}
beta of midpoint:\t{beta_value}
f of midpoint:\t{right_hand_side_value}
k values:\t{k_11}
d values:\t{d_1}
''')